In [ ]:
import pandas as pd
import os, time
import numpy as np
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_model_optimization as tfmot
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#EfficientNetb0
data_dir = "data/north_american_predators/greyscale_downsampled_split/train"
dataset_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/greyscale_downsampled_split/val"
dataset_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/greyscale_downsampled_split/test"
dataset_test = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)


Found 13823 files belonging to 8 classes.
Found 3454 files belonging to 8 classes.
Found 4317 files belonging to 8 classes.


In [ ]:
def evaluate_model_tflite(interpreter, dataset):
    #List of dictionaries
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    correct = 0
    total = 0

    for images, labels in dataset:
        input_type = input_details[0]['dtype']
        # Convert the batch to the correct type
        images = images.numpy().astype(input_type)
        labels = labels.numpy()
        
        for i in range(images.shape[0]):
            test_image = np.expand_dims(images[i], axis=0)
            #From Tensorflow API Docs
            interpreter.set_tensor(input_details[0]['index'], test_image)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            #Gets the most prominent class
            pred = np.argmax(output)
            if pred == labels[i]:
                correct += 1
            total += 1

    accuracy = correct / total
    return accuracy

In [15]:
#Issue with Keras 3.11. Must load weights after the fact
base_model = tf.keras.applications.EfficientNetB0(weights=None,
                                                input_shape=(224,224,3),
                                                include_top=False)
base_model.load_weights("efficientnetb0_notop.h5")
 
base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(8, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [16]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
early_stop = EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

start_time = time.time()
model.fit(dataset_train, epochs=100, validation_data=dataset_val, callbacks=[early_stop])
end_time = time.time()
training_duration = end_time - start_time
print(f"Training time: {training_duration:.2f} seconds")

Epoch 1/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 102s 457ms/step - loss: 0.3301 - sparse_categorical_accuracy: 0.9133 - val_loss: 0.3606 - val_sparse_categorical_accuracy: 0.8619
Epoch 2/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 95s 438ms/step - loss: 0.0905 - sparse_categorical_accuracy: 0.9760 - val_loss: 0.3297 - val_sparse_categorical_accuracy: 0.8796
Epoch 3/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 93s 431ms/step - loss: 0.0642 - sparse_categorical_accuracy: 0.9818 - val_loss: 0.3296 - val_sparse_categorical_accuracy: 0.8830
Epoch 4/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 92s 428ms/step - loss: 0.0508 - sparse_categorical_accuracy: 0.9858 - val_loss: 0.3265 - val_sparse_categorical_accuracy: 0.8856
Epoch 5/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 95s 440ms/step - loss: 0.0439 - sparse_categorical_accuracy: 0.9868 - val_loss: 0.3465 - val_sparse_categorical_accuracy: 0.8836
Epoch 6/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 93s 432ms/step - loss: 0.0389 - sparse_categorical_accuracy: 0.9884 - val_loss: 0.3514 - val_sparse_categori

In [17]:
start_time = time.time()
test_loss, test_acc = model.evaluate(dataset_test, verbose=0)
end_time = time.time()
training_duration = end_time - start_time
print(f"Evaluation time: {training_duration:.2f} seconds")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc*100:.2f}%")

Evaluation time: 23.00 seconds
Test Loss: 0.3455
Test Accuracy: 89.14%


In [18]:
model.save('model_EfficientNetb0_grey.keras')
print("Baseline model in Mb:", os.path.getsize(r'model_EfficientNetb0_grey.keras') / float(2**20))

Baseline model in Mb: 16.36735248565674


In [19]:
loaded_model = tf.keras.models.load_model(r'model_EfficientNetb0_grey.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('model_EfficientNetb0_quantized_grey.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpx6oy5nxo\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpx6oy5nxo\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmpx6oy5nxo'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_5')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2955461329168: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2955461328976: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2952125662224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2952125661264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2952125661456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2952125662416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2952125664336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2952125661648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2952125664528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2952125662800: TensorSpec(shape=(), dtype=tf.resource,

In [20]:
loaded_model = tf.keras.models.load_model(r'model_EfficientNetb0_grey.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()
with open('model_EfficientNetb0_grey.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpvui14q50\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpvui14q50\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmpvui14q50'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_5')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2955619763024: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2955619762640: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2955763679312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2955763680080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2955763681616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2955763682384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2955763682960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2955763681808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2955763683152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2955763682192: TensorSpec(shape=(), dtype=tf.resource,

In [ ]:
#EfficientNetb0 trained on greyscale images
interpreter = tf.lite.Interpreter(model_path="model_EfficientNetb0_grey.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)        
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_EfficientNetb0_grey.tflite') / float(2**20))

c:\Graduate_School\Fall_2025\ECE528\.venv\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


TFLite accuracy: 0.8913597405605744
TFlite model in Mb: 15.331218719482422


In [ ]:
#EfficientNetb0 quantized trained on greyscale images
interpreter = tf.lite.Interpreter(model_path="model_EfficientNetb0_quantized_grey.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_EfficientNetb0_quantized_grey.tflite') / float(2**20))

TFLite accuracy: 0.8897382441510308
TFlite model in Mb: 4.345817565917969
